<a href="https://colab.research.google.com/github/PavanGavit/FL_Lab/blob/main/FL_A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error



In [2]:
# Load dataset
data = pd.read_csv("/content/Housing - Housing.csv")

# Select numeric features only
features = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking']
target = 'price'

X = data[features].values
y = data[target].values


In [3]:
scaler = StandardScaler()
X = scaler.fit_transform(X)


In [4]:
num_clients = 5

X_splits = np.array_split(X, num_clients)
y_splits = np.array_split(y, num_clients)

client_datasets = list(zip(X_splits, y_splits))


In [5]:
def train_local_model(X, y, w, b, lr=0.01, epochs=20):
    n = len(y)
    for _ in range(epochs):
        y_pred = X @ w + b
        error = y_pred - y
        dw = (1/n) * X.T @ error
        db = (1/n) * np.sum(error)
        w -= lr * dw
        b -= lr * db
    return w, b




In [6]:
def fedavg(local_weights, local_biases, data_sizes):
    total_samples = sum(data_sizes)
    w_global = np.zeros_like(local_weights[0])
    b_global = 0.0

    for w, b, size in zip(local_weights, local_biases, data_sizes):
        w_global += (size / total_samples) * w
        b_global += (size / total_samples) * b

    return w_global, b_global


In [7]:
num_features = X.shape[1]
w_global = np.zeros(num_features)
b_global = 0.0

rounds = 25

for r in range(rounds):
    local_weights = []
    local_biases = []
    data_sizes = []

    for X_c, y_c in client_datasets:
        w_local, b_local = train_local_model(
            X_c, y_c,
            w_global.copy(),
            b_global
        )

        local_weights.append(w_local)
        local_biases.append(b_local)
        data_sizes.append(len(y_c))

    # Server aggregation
    w_global, b_global = fedavg(
        local_weights, local_biases, data_sizes
    )

    # Evaluate global model
    y_pred_global = X @ w_global + b_global
    mse = mean_squared_error(y, y_pred_global)

    print(f"Round {r+1:02d} | Global MSE: {mse:.2f}")


Round 01 | Global MSE: 18614986305204.66
Round 02 | Global MSE: 13393929796211.13
Round 03 | Global MSE: 9793533084355.97
Round 04 | Global MSE: 7303150641178.47
Round 05 | Global MSE: 5575983506877.56
Round 06 | Global MSE: 4375229822362.27
Round 07 | Global MSE: 3538509584607.12
Round 08 | Global MSE: 2954101115292.86
Round 09 | Global MSE: 2544928372147.79
Round 10 | Global MSE: 2257697238249.81
Round 11 | Global MSE: 2055485978486.95
Round 12 | Global MSE: 1912670773889.84
Round 13 | Global MSE: 1811439266630.04
Round 14 | Global MSE: 1739389068073.00
Round 15 | Global MSE: 1687870202527.89
Round 16 | Global MSE: 1650839084084.39
Round 17 | Global MSE: 1624065007214.23
Round 18 | Global MSE: 1604580001631.60
Round 19 | Global MSE: 1590296948041.48
Round 20 | Global MSE: 1579744177169.20
Round 21 | Global MSE: 1571880799456.52
Round 22 | Global MSE: 1565968045841.07
Round 23 | Global MSE: 1561479509163.18
Round 24 | Global MSE: 1558038430796.15
Round 25 | Global MSE: 1555373810555.4

In [8]:
print("\nFinal Global Model Parameters")
print("Weights:", w_global)
print("Bias:", b_global)



Final Global Model Parameters
Weights: [657722.41567923 159035.41965975 492469.34389751 403701.79235521
 315827.26362284]
Bias: 4686070.507299106
